In [27]:
# Warning control
import warnings
import os

warnings.filterwarnings("ignore")

from crewai import Agent, Task, Crew
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

- `MDXSearchTool` : this will help us to do semantic search in our resume and we can brag and make changes in it

In [28]:
from crewai_tools import FileReadTool, ScrapeWebsiteTool, MDXSearchTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path="Resume/sanketResume.md")
semantic_search_resume = MDXSearchTool(
    mdx="Resume/sanketResume.md",
)

## agents

In [29]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on " "job posting to help job applicants",
    tools=[scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
)

In [30]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
    "to help them stand out in the job market",
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
)

In [31]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a " "resume stand out in the job market.",
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
)

In [32]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
    "based on the resume and job requirements",
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
)

## Tasks

In [33]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True,
)

-- In the context of the job application system, setting `async_execution=True` for the profile task allows the profiler agent to work on compiling the comprehensive profile in the background while other tasks (like resume creation or cover letter writing) can proceed simultaneously. This is particularly useful when the task involves multiple external data sources or API calls that might have varying response times.

In [34]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True,
)

In [35]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="InterviewPrep/tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist,
)

In [36]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="InterviewPrep/interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer,
)

## Crew

In [37]:
job_application_crew = Crew(
    agents=[researcher, profiler, resume_strategist, interview_preparer],
    tasks=[
        research_task,
        profile_task,
        resume_strategy_task,
        interview_preparation_task,
    ],
    verbose=True,
)

2025-04-13 16:58:57,507 - 8234502208 - __init__.py-__init__:535 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

In [38]:
job_application_inputs = {
    "job_posting_url": "https://www.amazon.jobs/en/jobs/2951596/data-scientist-aws-security",
    "github_url": "https://github.com/sanket1105",
    "personal_writeup": """Sanket is a well accomplished Data Scienstist,
    He is working as a data scientist at Cox communications and has been working on interesting projects like 
    NLP and AI chatbots and customer churn model. On top of that he has also been working on areas like
    MLOPS, NLP, docker and all. He also as a Data Scientist masters degree from Stevens institute of technology.
    With all his experience, he would be great for all the DS roles which are less then 3 years of experience and are in
    NLP, MLOPS domain1""",
}

In [39]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://www.amazon.jobs/en/jobs/2951596/data-scientist-aws-security) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/sanket1105) URLs, and personal write-up (Sanket is a well accomplished Data Scienstist,
    He is working as a data scientist at Cox communications and has been working on interesting projects like 
    NLP and AI chatbots and customer churn model. On top of that he has also been working on areas like
    MLOPS, NLP, docker and all. He also as a Data Scientist masters degree from Stevens institute of technology.
    With all his experience, he

In [41]:
from IPython.display import Markdown, display

display(Markdown("InterviewPrep/tailored_resume.md"))

Sanket Zanwar
201-238-9319 | snaketzanwar1128@gmail.com | New York, USA | [LinkedIn](https://www.linkedin.com/in/sanket-zanwar/) | GitHub

**Education**
- **Master of Science in Data Science**, Stevens Institute of Technology, USA, May 2024
  - CGPA: 3.99/4.0
- **Bachelor of Technology in Electronics and Communication Engineering**, National Institute of Technology, Goa, India, June 2022
  - CGPA: 9.35/10.00

**Professional Summary**
Data Scientist with over 2 years of experience specializing in machine learning, data querying with SQL, and scripting with Python. Proven track record at Cox Communications, developing and deploying advanced ML models and achieving significant improvements in customer engagement and product performance. Adept at applying theoretical models in practical, high-stakes environments. Enthusiastic about leading teams and fostering inclusive, collaborative workplace cultures.

**Technical Skills**
- **Programming Languages**: Python, R, PySpark, SQL, C++
- **Packages and Tools**: GitHub, Docker, Power BI, Tableau, Pytorch, Databricks, Hadoop, Spark, Hive, AWS (S3, EC2, IAM, DataLake, AutoML, Athena, Sagemaker), Google Cloud (BigQuery), TensorFlow, Pandas, Numpy, GitLab, Keras, Excel, Salesforce, Redshift, Streamlit, Matplotlib, Yellowbrick

**Professional Experience**
- **Data Scientist, Cox Communications**, Atlanta, GA, May 2023 - Ongoing
  - Developed propensity-to-buy models, enhancing customer identification by 64%, and leading cross-functional teams to exceed performance targets.
  - Implemented machine learning models using Python and SQL, improving predictive accuracy and model reliability through innovative A/B testing and algorithm calibration.
  - Championed the use of AWS tools (Athena, Quicksight) to analyze data and deliver insights that reduced customer churn by 14% and improved campaign ROI by 22%.

- **Data Science Intern, Dataperformers**, April 2021 - June 2021
  - Optimized data retrieval and processing using AWS S3 and Snowflake, enhancing system efficiency and supporting advanced data analysis projects.

**Leadership and Mentorship**
- **Teaching Assistant and Grader for MA541: Statistical Methods**, Stevens Institute of Technology, January 2023 - May 2023
  - Instructed students on statistical methods, emphasizing practical application and collaborative learning, preparing the next generation of data scientists.

**Key Projects**
- **MLOps Repository**: Spearheaded the integration of MLOps practices, significantly enhancing model deployment efficiency and team productivity.
- **Time Series Analysis and Cryptocurrency Forecasting**: Utilized advanced statistical models to predict market trends, achieving industry-leading accuracy.
- **Quora Question Pair Similarity**: Applied NLP techniques to improve machine learning model performance for text analysis.

**Interests**
Passionate about continuous learning and applying the latest advancements in AI/ML to solve real-world problems. Committed to contributing to diversity and inclusion in the tech community.

**Professional Philosophy**
Believes in leveraging data-driven insights and advanced technology to enhance business outcomes and human capabilities, fostering an environment of continuous improvement and innovation.

**Additional Activities**
Active participant in ongoing DEI events and dedicated to personal and professional growth through continuous learning and training opportunities in the field of data science and machine learning.